In [1]:
import lkml
import os

In [2]:
lookml_base = 'C:\\Users\\EricHeidbreder\\Documents\\looker_projects\\a8_case_study\\'
lookml_folder = "_layers"
lookml_filename = "_base.layer.lkml"
lookml_path = os.path.join(lookml_base, lookml_folder, lookml_filename)

In [4]:
def generate_new_filename(lookml_path, extension):
    lookml_filename_new = lookml_path.split('\\')[-1].split('.')
    lookml_filename_new[0] += extension
    return '.'.join(lookml_filename_new)

In [5]:
def find(lst, key, value):
    '''
    Finds a value in a list of dictionaries
    '''
    for i, dic in enumerate(lst):
        if dic[key] == value:
            return i
    return f'Value "{value}" not found'

In [123]:
def generate_sets(lookml_path):
    '''
    Automatically generate sets for each dimension group in a view
    '''
    # Indicates if a change was made to the file
    made_change = 0

    with open(lookml_path, 'r+') as file:
        result = lkml.load(file)

    try:    
        view_list = result['views']
    except:
        return 'No views in current file'


    for view_num in range(len(view_list)):

    # # get the view number index from list of dictionaries
    # view_num = find(view_list, 'name', view_name)
    
        # Are there any dimension groups in this view?
        try:
            dim_group_count = len(view_list[view_num]['dimension_groups'])

        except:
            print(f'No dimension groups in {os.path.basename(lookml_path)} view')
            continue

        # Yay! There's at least one dimension group.
        # Now, are there any sets in this dimension group?
        try:
            current_set_list = view_list[view_num]['sets']

        # There weren't any sets, but that's okay. We'll create a new one.
        except:
            current_set_list = view_list[view_num]['sets'] = []

        # Creating some tracking info
        new_set_names = []

        # If the duration/time fields got left out, they will default to these
        default_duration_fields = [
            'days',
            'hours',
            'minutes',
            'months',
            'quarters',
            'seconds',
            'weeks',
            'years'
        ]

        default_time_fields = [ 
            'date',
            'month',
            'quarter',
            'time',
            'week',
            'year'     
        ]

        # Iterate through dimension groups and generate sets based on the
        # timeframes in the dimension group
        for dim_group in view_list[view_num]['dimension_groups']:

            # Currently can't handle dynamic label names, so if there's
            # any liquid, we'll skip for now
            try:
                if "{%" in dim_group['sql']:
                    continue
                    print("There's Liquid!")
            except:
                pass

            # extracting info from the dimension group and creating
            # variables to be used in set generation
            try:
                dim_group_type = dim_group['type']
            except:
                print('No type, must be a refinement...continuing.')
                continue

            dim_group_name = dim_group['name']

            try:
                dim_group_timeframes = dim_group['timeframes']
            except:
                if dim_group_type == 'time':
                    dim_group_timeframes = default_time_fields
                elif dim_group_type == 'duration':
                    dim_group_timeframes = default_duration_fields

            new_set_name = f'{dim_group_name}_{dim_group_type}_fields'

            # there are different naming formats for duration vs. time types, so we need to build different field name
            # lists for the different types
            if dim_group_type == 'time':
                field_names = [dim_group_name + '_' + timeframe for timeframe in dim_group_timeframes]
            elif dim_group_type == 'duration':
                field_names = [timeframe + '_' + dim_group_name for timeframe in dim_group_timeframes]

            view_list[view_num]['sets'].append(
                {'fields': field_names,
            'name': new_set_name}
            )

            new_set_names.append(new_set_name)

            made_change = 1

    # We only want to overwrite the file if we made any changes

    if made_change:

        print(f'Created the following {dim_group_count} new sets:')
        [print(set_name) for set_name in new_set_names]

        orig_lookml_filename = lookml_path.split('\\')[-1]

        new_lookml_filename = generate_new_filename(lookml_path, '_new')

        lookml_new_path = lookml_path.replace(orig_lookml_filename, new_lookml_filename)

        with open(f'{lookml_path}', 'w+') as new_file:
            lkml.dump(result, new_file)
    

In [124]:
for folder in os.listdir(lookml_base):
    # First, let's only keep folders that don't begin with periods and make sure we're not running any stray .lkml files through this
    if folder[0] != '.' and folder.split('.')[-1] not in ['lkml']:
        for filename in os.listdir(lookml_base + folder):
            # ignore any files beginning with a period, and any python or jupyter notebooks files (py or ipynb)
            if filename[0] != '.' and filename.split('.')[-1] not in ['py', 'ipynb']:
                lookml_path = os.path.join(lookml_base, folder, filename)
                generate_sets(lookml_path)

Created the following 1 new sets:
creation_to_delivery_duration_fields
No dimension groups in location.layer.lkml view
No dimension groups in location.layer.lkml view
No dimension groups in profit.layer.lkml view
No dimension groups in revenue.layer.lkml view
No dimension groups in user_age_and_name.layer.lkml view
No dimension groups in order_sequence_1.layer.lkml view
No dimension groups in order_sequence_1.layer.lkml view
No dimension groups in order_sequence_1.layer.lkml view
No dimension groups in order_sequence_2.layer.lkml view
No dimension groups in order_sequence_2.layer.lkml view
No dimension groups in profit_per_order.layer.lkml view
No dimension groups in profit_per_order.layer.lkml view
No dimension groups in delivery_duration.group.lkml view
No dimension groups in demographics.group.lkml view
No dimension groups in ids.group.lkml view
No dimension groups in ids.group.lkml view
No dimension groups in location.group.lkml view
No dimension groups in location.group.lkml view
